## Goal
* Gather a table of sunset and sunrise times from an array of locations

https://sunrise-sunset.org/api

In [160]:
# Import tools
from urllib.request import urlopen
from datetime import timedelta, date, datetime
from dateutil import tz
import json
import pandas as pd

## Build Date Array

In [161]:
def daterange(start_date, end_date):
    for n in range(int ((end_date - start_date).days)):
        yield start_date + timedelta(n)

# Initialize date range
start_date = date(2017, 7, 1)
end_date = date(2017, 11, 1)

dates = []

for single_date in daterange(start_date, end_date):
        dates.append(single_date.strftime("%Y-%m-%d"))

In [66]:
# display sample
for date in dates[-4:]:
    print (date)

2017-10-28
2017-10-29
2017-10-30
2017-10-31


## Load Location Dictionary

In [68]:
# static location selection
location = 'missoula'
lat = '46.872'
lon = '113.9940'

In [85]:
# load location df
stations = pd.read_csv('../../output/weather_monitoring-locations.csv')

In [87]:
# display sample
stations.loc[:1]

,id,lat,lon,neighborhood,city
0,KMTCORVA9,46.277557,-114.045059,Hamilton Heights,Corvallis
1,KMTCORVA10,46.284927,-114.106499,All View Lane,Corvallis


## Build URLs

In [139]:
stations.loc[0]

id                     KMTCORVA9
lat                      46.2776
lon                     -114.045
neighborhood    Hamilton Heights
city                   Corvallis
Name: 0, dtype: object

In [155]:
def build_url(dates): 
    urls = []
    # loop through stations in weather_monitoring-locations.csv
    for index, row in stations.iterrows():
        # loop through date range set above
        for date in dates:
            lat = row['lat']
            lon = row['lon']
            url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&lng=' + str(lon) + '&date=' + date
            urls.append(url)
    return(urls)

In [156]:
url_batch = build_url(dates)

In [158]:
print(url_batch[0])

https://api.sunrise-sunset.org/json?lat=46.277557&lng=-114.045059&date=2017-07-01


In [159]:
len(url_batch)

4182

## Scratch Work Below

In [82]:
# url queries the sunrise-sunset.org api

test_url = 'https://api.sunrise-sunset.org/json?lat=' + str(lat) + '&' + str(lon) + '&date=' + selected_date

In [83]:
test_url

'https://api.sunrise-sunset.org/json?lat=46.364567&-114.242287&date=2017-07-18'

In [103]:
def api_call(url):
    f = urlopen(url)
    json_string = f.read() 
    parsed_json = json.loads(json_string)
    return parsed_json

In [31]:
results = api_call(url)

NameError: name 'api_call' is not defined

In [105]:
# Add date and location to dictionary

results['results']['date'] = selected_date
results['results']['location'] = location
results['results']['lat'] = lat
results['results']['lon'] = lon

In [106]:
print('* All times UTC *\n')

for key, value in results['results'].items():
    print(key + ": " + value)

* All times UTC *

sunrise: 8:48:14 PM
sunset: 12:12:23 PM
solar_noon: 4:30:18 AM
day_length: 15:24:09
civil_twilight_begin: 8:10:55 PM
civil_twilight_end: 12:49:41 PM
nautical_twilight_begin: 7:21:40 PM
nautical_twilight_end: 1:38:56 PM
astronomical_twilight_begin: 6:18:01 PM
astronomical_twilight_end: 2:42:35 PM
date: 2017-07-18
location: missoula
lat: 46.872
lon: 113.9940
